# D-STACK: High Throughput DNN Inference by Effective Multiplexing and Spatio-Temporal Scheduling of GPUs
**Идея:** Создание framework, который позволяет насаживать несколько моделей на 1 видеокарту, при этом при inference не будет сильно много из-за этого проблем с задержками inference каждом модели [ссылка](https://arxiv.org/pdf/2304.13541)

![ExampleProblem](./Pictures/D-STACK/ExampleProblem.png)

## Modeling DNN parallelism

###  Compute Bound vs. Memory Bound Workloads

Задержка доступа к параметрам и весам слоя DNN из памяти DRAM графического процессора может быть значительной. Многие исследования предположили, что привязанные к памяти ядра DNN могут иметь небольшой объем вычислений и, вероятно, будут ограничены пропускной способностью памяти графического процессора. NVIDIA предложила метрику арифметической интенсивности (A.int)  для оценки того, является ли ядро ограниченным памятью или вычислительными ресурсами. A. int ядра вычисляется как отношение операций с плавающей запятой к памяти (байтам), которую оно получило. т.е. $A.int =\frac{operations}{bytes}$  . NVIDIA сообщает, что арифметический индекс графического процессора V100 (в нашем тестовом стенде) составляет 139,8 FLOPS/байт. Любое ядро с меньшим арифметическим индексом графического процессора ограничено памятью, в то время как ядро с более высоким индексом ограничено вычислениями.

![ExamplesComputeMemory](./Pictures/D-STACK/ExamplesComputeMemory.png)

Большинство слоев свертки превышают A.int графического процессора, поэтому они привязаны к вычислительным ресурсам. Эти уровни могут сократить время выполнения, если доступно больше вычислительных ресурсов. Тем не менее, ядра, такие как LSTM в GNMT, которые работают с большими входными и выходными функциями (1024 функции в GNMT), требуют много данных, но выполняют относительно меньше вычислений по сравнению со сверткой. Поэтому они набирают очень низкие A.int. Следует отметить, что DNN не полностью состоят из сверточных или LSTM-слоев. Однако СНС, как правило, имеют больше ядер свертки.

**Замечание:** [Статья от Nvidia о том, как это все расчитывается](https://docs.nvidia.com/deeplearning/performance/dl-performance-gpu-background/index.html)

### Data Modelimg

Теперь мы моделируем аналитическую модель DNN, которая демонстрирует характеристики большинства актуальных моделей DNN с точки зрения различий в вычислительной рабочей нагрузке между различными ядрами. Мы моделируем DNN, состоящую из нескольких последовательных ядер, выполняемых на GPU (и других ускорителях), а не слоев, как это часто используется в других исследованиях машинного обучения. Используя профилирование NVPROF, мы заметили, что каждый слой (например, слой свертки) часто реализуется как комбинация нескольких ядер в GPU, таким образом, мы используем ядро в качестве основного компонента выполнения DNN в этой модели. Модель определяет наилучшую рабочую точку (колено) GPU% для DNN. В нашей модели мы разбиваем рабочую нагрузку DNN на распараллеливаемые операции (вычислительные задачи), операции чтения и записи памяти, а также сериализованные (нераспараллеливаемые) операции и наблюдаем за эффектом изменения ресурсов графического процессора. Несмотря на то, что наша модель проста, она фиксирует все накладные расходы системного уровня, которые способствуют задержке DNN, и предоставляет нам хорошую аппроксимацию колена каждой модели. Простота модели дополнительно помогает в оценке DNN в различных графических процессорах с разным числом SM, а также в другом оборудовании ускорителя.

Выбранная нотация, используемая в анализе, показана в таблице ниже. Как и в типичных графических процессорах, каждый из S SM, выделенных DNN, будет обрабатывать одну параллельную операцию за $t_p$ времени. С точки зрения моделирования, мы упорядочиваем ядра по объему вычислений без потери общности. DNN имеют произвольный порядок выполнения ядра. Однако колено модели зависит от пиковых требований к вычислениям ядер, а не от порядка выполнения каждого ядра.

![Annotation](./Pictures/D-STACK/Annotation.png)

Мы устанавливаем первое ядро $K_1$ как ядро с наибольшим количеством распараллеливаемых операций $N_1$, которое для целей моделирования выбирается как $N_1 = p * b$. Для последующих ядер рабочая нагрузка уменьшается на фиксированную величину, так что $N_i > N_{i+1}$. Формула (1) определяет количество распараллеливаемых операций для каждого ядра в DNN. Уменьшаем количество распараллеливаемых задач на фиксированную величину, $\frac{p*b}{K_{max}}$,

$$
\begin{equation*}
N_i = 
 \begin{cases}
    p*b \ i = 1 \\ 
    \left[ N_{i-1} - \frac{p*b}{K_{max}} \right] \ i  > 1 \\
 \end{cases}
\end{equation*} \ (1)
$$

для каждого последующего ядра. Количество параллельных операций уменьшается и достигает ∼ 0 для последнего $(K_{max})$ ядра. Соответственно, мы определяем общее время выполнения для распараллеливаемых задач каждого ядра как $W_i = N_I * t_p$

Примечание: В идеале, $W_i$ потенциально может быть завершено за $T_P$ единиц времени, когда мы выделяем больше или равно $N_I$ SM для выполнения $W_i$. Если мы примем во внимание, что аппаратное обеспечение GPU способно обеспечить S SMs для выполнения K_i, то, без потери общности, мы можем показать, что время, затраченное на завершение обработки ядра, будет зависеть от минимума присущего ему параллелизма, определенного как $N_i$, и от количества SM, выделенных для выполнения операции. Таким образом, время выполнения распараллеливаемых операций в каждом ядре DNN может быть вычислено с помощью формулы (2) для каждого ядра:

$$
E_i = \frac{W_i}{\max(1, \min(S, N_i))} \ (2)
$$

В DNN часто повторяются ядра во время вывода DNN. Мы определяем количество повторений ядра как  $R_i$. Затем мы учитываем время, затраченное на выполнение всех сериализованных операций, в том числе на запуск ядра и ожидание данных ядром. Время начала работы ядра считается постоянным, $t_{np}$, для каждого слоя. Время ожидания данных ядром, однако, зависит от входных данных и параметров ядра. Каждое ядро DNN имеет определенный объем данных (параметры модели, входные данные), которые должны быть получены из GPU DRAM (основной/глобальной памяти GPU) к ядрам CUDA в SM. Мы заметили, что общая глобальная пропускная способность чтения/записи памяти увеличивается пропорционально количеству выделенных SM. Мы определяем задержку для каждого ядра, вызванную ожиданием ядром загрузки параметров, входных данных и других данных по формуле (3). Таким образом, общее время нераспараллеливаемых (последовательных) операций $W_{se}$ можно вычислить по формуле (4). Мы используем формулы (2) и (4) для вычисления времени выполнения DNN, $E_t$ как в уравнении (5)

$$
E_m = \frac{d_i * S}{M} \ (3) \\ 
W_{se} = b * \sum\limits_{i=1}^{K_{max}}R_i * (t_{np} + E_m) \ (4) \\ 
E_t = W_{se} + \sum\limits_{i=1}^{K_{max}}R_iE_i \ (5)
$$

Теперь мы моделируем общее время выполнения DNN в различных условиях, т. е. варьируя количество распараллеливаемых и нераспараллеливаемых операций в каждом ядре и количество SM в графическом процессоре. Как и в типичных графических процессорах, мы предполагаем, что количество SM, выделенных для DNN, остается неизменным. На рисунке ниже показано влияние на время выполнения DNN при назначении разного количества SM. Сначала мы создали DNN с 50 ядрами, т.е. $K_{max}= 50$. Мы установили время, необходимое для параллельной операции, $t_p=40$ единицам, и для сериализованных операций — $t_{np} = 10$ единицам. Повторяем симуляцию для 3-х случаев: $N_1 = 60, 40, 20$

Во всех трех случаях время выполнения очень велико, когда количество SM невелико (от 1 до 5 SM), что отражает штраф в виде нехватки ресурсов для присущей ему степени параллелизма при выполнении ядра DNN. Однако по мере увеличения количества SM задержка выполнения уменьшается. Интересно (см. увеличенную часть рисунка 4а), что в каждом из сценариев наступает момент, когда выдача большего количества SM за одну точку не приводит к дальнейшему уменьшению задержки. Когда количество подготовленных SM превышает объем параллелизма, присущего ядру DNN, дальнейшее уменьшение задержки не происходит. Еще до того, как достигнуть этой точки, увеличение задержки за счет увеличения числа SM достигает точки убывающей отдачи1 . Мы стремимся найти наиболее эффективное количество SM ( ), необходимое для выполнения данной DNN, чтобы максимально использовать выделенные SM. Чтобы вычислить это, нам нужно найти максимум $\frac{1}{E_t S}$, который представляет собой работу DNN, обработанную в единицу времени на SM. Для этого мы дифференцируем это по времени, затраченного на выполнение DNN. 

$$
\frac{d}{E_t}\left( \frac{1}{E_t S} \right) = -\frac{1}{E_t^2 S} \ (6)
$$

На рисунке ниже показана эта производная первого порядка обратной задержки (формулы (6) ), показывающая, что SM для $N_1 = 20, 40, 60$ достигает максимума при 9, 24 и 31 SM соответственно. Следовательно, работа в этой производной «максимальной» точке для DNN гарантирует, что существует достаточное количество SM для обеспечения низкой задержки при достижении наиболее эффективного использования SM. Более того, из этого мы видим, что «максимум» достигает пика при гораздо более низком SM, чем соответствующее значение $N_1$. Это связано с влиянием выполнения сериализованных задач рядом с распараллеливаемыми задачами. Это приводит к снижению (или отсутствию) использования многих выделенных SM для сериализованных задач. Таким образом, дальнейшее снижение задержки за счет увеличения SM является минимальным.

![MOdeling](./Pictures/D-STACK/Modeling.png)

##  Optimal Batching for DNNs

Пакетная обработка — это компромисс между повышением пропускной способности и более высокой задержкой. Вывод пакета запросов требует больше вычислений, что увеличивает время вывода. Подготовка большего пакета, т. е. прием и передача данных из сети на графический процессор, также вносит дополнительную задержку. Предоставление более высокого % графического процессора для большего пакета может уменьшить увеличение задержки вывода. Тем не менее, предоставление большего процента графического процессора может быть расточительным. Мы используем метрику:

$$
Efficacy(\eta) = \frac{Thoughput}{Latency * GPU\% } \ (7)
$$

Мы определяем $\eta$ как DNN при определенном размере партии. Эффективность, позволяет нам узнать, какую пропускную способность графический процессор производит в единицу времени на единицу ресурса графического процессора (GPU%).

Мы профилировали модель ResNet-50 для инференса при различных размерах партии и конфигурации GPU%. На рисунке ниже показано, что как очень высокий, так и очень низкий размер партии приводит к низкой эффективности из-за высокой задержки и сниженной пропускной способности соответственно, таким образом, требуется оптимальный размер партии. Теперь мы разрабатываем формулировку оптимизации, которая может предоставить нам правильный размер партии и GPU% для модели с учетом крайнего срока.

![batch problem](./Pictures/D-STACK/BatchProblem.png)


###  Optimum Batch Size for Inference
Во-первых, мы представляем ключевые обозначения, используемые для оптимизации в таблице ниже

![Table](./Pictures/D-STACK/BatchAnnotation.png)

Размер пакета является произведением средней скорости входящих запросов на время сборки запроса: $b_i = Request-Rate * C_i$. Throughput $T_i$ - количество изображений, выводимых за единицу времени (формула 8. Зная пропускную способность (формула 8), мы можем записать (формулу 7) как формулу 9). Формула 9 имеет ту же форму, что и первая производная обратной задержки, формула 6.

$$
T_i = \frac{b_i}{f_L(p_i, b_i)} \ (8) \\ 
\eta = \frac{b_i}{(f_l(p_i,b_i))^2 * GPU\% } \ (9)
$$

Мы стремимся максимизировать $Efficacy(\eta)$, чтобы получить наилучший баланс параметров на основе ограничений 10, 11 и 12. Ограничения выражают следующие требования: неравенство 10 -  размер партии должен быть меньше или равен максимальному размеру партии модели может принять, уравнение 11 - сумма времени, затраченного на агрегацию пакета через network($C_i$) и выполнение его вывода, который должен удовлетворять SLO. Уравнение 12: При работе с высоким процентом запросов мы можем регулярно собирать большие партии для логического вывода. Однако запрос, который не может быть включен в текущий пакет из-за ограничения уравнения 11, должен быть выведен в следующем пакете. Тогда крайний срок для следующего пакета — это крайний срок для самого старого ожидающего запроса. Поэтому мы следим за тем, чтобы SLO в два раза превышал время, необходимое для выполнения пакета.

$$
1 \leq b_i \leq  MaxBatchSize \ (10) \\
f_L(p_i, b_i) + C_i \leq SLO_i \ (11) \\
f_L(p_i, b_i) \leq \frac{SLO_i}{2}
$$

**Замечание:**
SLO (Service Level Objective) здесь означает целевой уровень обслуживания или целевое время выполнения определённой задачи, которое необходимо достичь.
В контексте данного текста SLO используется как ограничение времени (в миллисекундах), которое определяет, сколько максимально времени можно потратить на выполнение двух процессов:
1. Аггрегацию запросов в батч через сеть
2. Исполнение (inference) модели на этом батче 

## GPU Scheduling of DNN models

### Scheduling with varying SLO
**Надо разобраться, пока знаний не хватило**

## D-STACK in Multi-GPU Clusters

Мы оценили D-STACK в кластере с несколькими графическими процессорами из 4 графических процессоров NVIDIA T4, каждый из которых имеет 40 SM (меньше V100) и 16 ГБ памяти. Мы использовали 4 различные модели машинного зрения: Mobilenet, Alexnet, ResNet-50 и VGG-19 (% графического процессора колена отличается для графического процессора T4 и V100). Мы сравниваем пропускную способность 3 различных сценариев мультиплексирования и планирования. Во-первых, мы предоставляем один графический процессор T4 исключительно для каждой модели DNN. Во втором сценарии мы размещаем все 4 модели в каждом графическом процессоре, временно деля графический процессор. Наконец, мы оцениваем D-STACK с помощью 4 моделей DNN. 

На рисунке ниже  показано, что временное планирование имеет почти такую же пропускную способность, как и каждая модель с эксклюзивным графическим процессором. Это происходит из-за недостаточного использования графического процессора моделями DNN. D-STACK имеет гораздо более высокую пропускную способность для каждой модели, с общей пропускной способностью на 160% выше, чем при временном обмене.

Общая пропускная способность вывода значительно увеличивается, так как кластер с несколькими графическими процессорами лучше используется D-STACK.

![MultiGPU](./Pictures/D-STACK/MultiGPU.png)


